In [1]:
#Import dependencies

import pandas as pd
import numpy as np

In [2]:
#Select files

students_file = 'students_complete.csv'
schools_file = 'schools_complete.csv'

In [3]:
#Import as dataframes
students_df = pd.read_csv(students_file)
schools_df = pd.read_csv(schools_file)

In [4]:
#Merge on school_name (like column)
school_data_all = pd.merge(students_df, schools_df, how='left', on=['school_name', 'school_name'])
school_data_all.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

Calculate the total number of schools

Calculate the total number of students

Calculate the total budget

Calculate the average math score 

Calculate the average reading score

Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

Calculate the percentage of students with a passing math score (70 or greater)

Calculate the percentage of students with a passing reading score (70 or greater)

Create a dataframe to hold the above results

Optional: give the displayed data cleaner formatting

In [5]:
#Calculate the total number of schools
total_schools = school_data_all.school_name.nunique()
# print('# schools: ' + str(total_schools))

#Calculate the total number of students
total_students = school_data_all.student_name.count()
# print('# students: ' + str(total_students))

# Calculate the total budget
budget_list = school_data_all.drop_duplicates(['school_name','budget']).groupby('school_name').agg({'budget':'sum'})
total_budget = budget_list.budget.sum()
# print('Total budget: $' + str(total_budget))

# Calculate the average math score 
math_mean = school_data_all.math_score.mean()
# print('Avg math: ' + str(math_mean))

# Calculate the average reading score
reading_mean = school_data_all.reading_score.mean()
# print('Avg reading: ' + str(reading_mean))

# Calculate the percentage of students with a passing math score (70 or greater)
math_passing_count = (school_data_all[school_data_all.math_score >= 70]).math_score.count()
math_passing_pct = math_passing_count/total_students * 100
# print('Math pass: ' + str(math_passing_pct))

# Calculate the percentage of students with a passing reading score (70 or greater)
reading_passing_count = (school_data_all[school_data_all.reading_score >= 70]).reading_score.count()
reading_passing_pct = reading_passing_count/total_students * 100
# print('Reading pass: ' + str(reading_passing_pct))

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
passing_rate = (math_mean + reading_mean)/2
# print('Passing rate: ' + str(passing_rate))

# Create a dataframe to hold the above results
district_data = {
    'Total Schools':[total_schools],
    'Total Students': [total_students],
    'Total Budget': [total_budget],
    'Avergage Math Score': [math_mean],
    'Average Reading Score': [reading_mean],
    '% Passing Math': [math_passing_pct],
    '% Passing Reading': [reading_passing_pct],
    '% Overall Passing Rate': [passing_rate]
}

district_summary = pd.DataFrame(district_data)
district_summary

# Optional: give the displayed data cleaner formatting

,Total Schools,Total Students,Total Budget,Avergage Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


# School Summary


Create an overview table that summarizes key metrics about each school, including:

School Name
School Type
Total Students
Total School Budget
Per Student Budget
Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)

In [6]:
#Get count of student names column to use to perform final calculations with
school_groups = school_data_all.groupby(['school_name'])

school_groups_counts = school_groups.count()
school_groups_counts = school_groups_counts.add_suffix('_count')
school_groups_counts.drop(['Student ID_count','gender_count','grade_count','School ID_count','reading_score_count', 'math_score_count','size_count','budget_count','type_count'], axis=1, inplace=True)

In [7]:
#Create sum columns 
school_groups_sums = school_groups.sum()
school_groups_sums.drop(['Student ID', 'School ID', 'size', 'budget'], axis=1, inplace=True)
school_groups_sums = school_groups_sums.add_suffix('_sum')

In [8]:
#Merge counts and sums to get table to perform calculations on
sums_and_counts = pd.merge(school_groups_counts, school_groups_sums, how='outer', left_index=True, right_on=['school_name'])

In [9]:
# Merge with schools_df - already have budget totals by school, so no need to recalculate
all_sums_and_counts = pd.merge(sums_and_counts, schools_df, left_index=True, right_on='school_name')
all_sums_and_counts.set_index('school_name', inplace=True)

In [10]:
# % Passing Math and % Passing Reading
school_data_all.index
passing_data = school_data_all.iloc[:,[4,5,6]]

passing_math_data = passing_data[passing_data['math_score'] >= 70].groupby('school_name')['math_score'].count()
passing_math_counts = pd.DataFrame(passing_math_data)
passing_math_counts = passing_math_counts.add_suffix('_counts')

passing_reading_data = passing_data[passing_data['reading_score'] >= 70].groupby('school_name')['reading_score'].count()
passing_reading_counts = pd.DataFrame(passing_reading_data)
passing_reading_counts = passing_reading_counts.add_suffix('_counts')

In [11]:
# Join dataframes to get one table with count of students who passed math and count of students who passed reading
total_passing_counts = passing_math_counts.join(passing_reading_counts)

In [12]:
# Merge to create master dataframe for school summary
school_summary_data = pd.merge(all_sums_and_counts, total_passing_counts, how='outer', left_index=True, right_on=['school_name'])

In [13]:
#Per Student Budget
school_summary_data['Per Student Budget'] = school_summary_data.budget/school_summary_data.student_name_count

# Average Math Score
school_summary_data['Average Math Score'] = school_summary_data.math_score_sum/school_summary_data.student_name_count

# Average Reading Score
school_summary_data['Average Reading Score'] = school_summary_data.reading_score_sum/school_summary_data.student_name_count

# % Passing Math
school_summary_data['% Passing Math'] = school_summary_data.math_score_counts/school_summary_data.student_name_count * 100

# % Passing Reading
school_summary_data['% Passing Reading'] = school_summary_data.reading_score_counts/school_summary_data.student_name_count * 100

# Overall Passing Rate (Average of the above two)
school_summary_data['Overall Passing Rate'] = (school_summary_data['% Passing Math'] + school_summary_data['% Passing Reading'])/2

# For final dataframe, 
school_summary_final = school_summary_data.drop(['reading_score_sum', 'math_score_sum', 'School ID', 'math_score_counts',
                                                 'reading_score_counts'], axis=1)
school_summary_final = school_summary_final[['type','student_name_count','budget','Per Student Budget','Average Math Score',
                                            'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate']]
school_summary_final = school_summary_final.rename(columns={'type': 'School Type', 'student_name_count': 'Total Students',
                                                          'budget':'Total School Budget'})
school_summary_final.index.names = ['School Name']
school_summary_final

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992


#  Top Performing Schools (By Passing Rate)
 
Sort and display the top five schools in overall passing rate

In [14]:
school_summary_final.sort_values('Overall Passing Rate', ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


#  Bottom Performing Schools (By Passing Rate)
 
Sort and display the five worst-performing schools

In [15]:
school_summary_final.sort_values('Overall Passing Rate', ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


# Math Scores by Grade
 
Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
Create a pandas series for each grade. Hint: use a conditional statement.
Group each series by school
Combine the series into a dataframe
Optional: give the displayed data cleaner formatting

In [16]:
# Create dataframe which only has school name, student id, math scores, and grade
math_data_by_grade = school_data_all.iloc[:,[0,3,4,6]]

# Pivot data
avg_math_by_grade = math_data_by_grade.reset_index().pivot_table(values='math_score', 
                                                                          index='school_name', 
                                                                          columns='grade', 
                                                                          aggfunc='mean')
avg_math_by_grade = avg_math_by_grade[['9th', '10th', '11th', '12th']]
avg_math_by_grade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


# Reading Score by Grade
 
Perform the same operations as above for reading scores

In [17]:
# Create dataframe which only has school name, student id, reading scores, and grade
reading_data_by_grade = school_data_all.iloc[:,[0,3,4,5]]

# Pivot data
avg_reading_by_grade = reading_data_by_grade.reset_index().pivot_table(values='reading_score', 
                                                                          index='school_name', 
                                                                          columns='grade', 
                                                                          aggfunc='mean')
avg_reading_by_grade = avg_reading_by_grade[['9th', '10th', '11th', '12th']]
avg_reading_by_grade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


#  Scores by School Spending
 
Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:

-- 1) Average Math Score
-- 2) Average Reading Score
-- 3) % Passing Math
-- 4) % Passing Reading
-- 5) Overall Passing Rate (Average of the above two)

In [26]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [27]:
school_summary_final['Spending Ranges (Per Student)'] = pd.cut(school_summary_final['Per Student Budget'],spending_bins,labels=group_names)
school_summary_final

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Bin,Spending Ranges (Per Student)
School Name,,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,$615-645,$615-645
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,<$585,<$585
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-645,$615-645
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,$615-645,$615-645
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-645,$615-645
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-675,$645-675
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,<$585,<$585
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-675,$645-675
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,$645-675,$645-675


In [28]:
schools_by_bin_df = school_summary_final.drop(school_summary_final.columns[[0, 1, 2, 3]], axis=1)
schools_by_bin_df = schools_by_bin_df.reset_index(drop=True)
schools_by_bin_df.set_index('Spending Ranges (Per Student)', inplace=True)
schools_by_bin_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Bin
Spending Ranges (Per Student),,,,,,
$615-645,77.048432,81.033963,66.680064,81.933280,74.306672,$615-645
<$585,83.061895,83.975780,94.133477,97.039828,95.586652,<$585
$615-645,76.711767,81.158020,65.988471,80.739234,73.363852,$615-645
$615-645,77.102592,80.746258,68.309602,79.299014,73.804308,$615-645
$615-645,83.351499,83.816757,93.392371,97.138965,95.265668,$615-645
$645-675,77.289752,80.934412,66.752967,80.862999,73.807983,$645-675
<$585,83.803279,83.814988,92.505855,96.252927,94.379391,<$585
$645-675,76.629414,81.182722,65.683922,81.316421,73.500171,$645-675
$645-675,77.072464,80.966394,66.057551,81.222432,73.639992,$645-675


In [29]:
schools_binned_final = schools_by_bin_df.groupby(['Spending Ranges (Per Student)'])
schools_binned_final.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


#  Scores by School Size
 
Perform the same operations as above, based on school size.

In [83]:
school_summary_final

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Bin
School Name,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,$615-645
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,<$585
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-645
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,$615-645
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-645
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-675
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,<$585
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-675
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,$645-675


In [30]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [31]:
school_summary_final['School Size'] = pd.cut(school_summary_final['Total Students'],size_bins,labels=size_group_names)
school_summary_final

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Bin,Spending Ranges (Per Student),School Size
School Name,,,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,$615-645,$615-645,Large (2000-5000)
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,<$585,<$585,Medium (1000-2000)
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-645,$615-645,Large (2000-5000)
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,$615-645,$615-645,Large (2000-5000)
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-645,$615-645,Medium (1000-2000)
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-675,$645-675,Large (2000-5000)
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,<$585,<$585,Small (<1000)
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-675,$645-675,Large (2000-5000)
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,$645-675,$645-675,Large (2000-5000)


In [32]:
schools_by_size_df = school_summary_final.drop(school_summary_final.columns[[0, 1, 2, 3]], axis=1)
schools_by_size_df = schools_by_size_df.reset_index(drop=True)
schools_by_size_df.set_index('School Size', inplace=True)
schools_by_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Bin,Spending Ranges (Per Student)
School Size,,,,,,,
Large (2000-5000),77.048432,81.033963,66.680064,81.933280,74.306672,$615-645,$615-645
Medium (1000-2000),83.061895,83.975780,94.133477,97.039828,95.586652,<$585,<$585
Large (2000-5000),76.711767,81.158020,65.988471,80.739234,73.363852,$615-645,$615-645
Large (2000-5000),77.102592,80.746258,68.309602,79.299014,73.804308,$615-645,$615-645
Medium (1000-2000),83.351499,83.816757,93.392371,97.138965,95.265668,$615-645,$615-645
Large (2000-5000),77.289752,80.934412,66.752967,80.862999,73.807983,$645-675,$645-675
Small (<1000),83.803279,83.814988,92.505855,96.252927,94.379391,<$585,<$585
Large (2000-5000),76.629414,81.182722,65.683922,81.316421,73.500171,$645-675,$645-675
Large (2000-5000),77.072464,80.966394,66.057551,81.222432,73.639992,$645-675,$645-675


In [33]:
schools_size_final = schools_by_size_df.groupby(['School Size'])
schools_size_final.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


# Scores by School Type
 
Perform the same operations as above, based on school type.

In [34]:
school_summary_final.columns

Index(['School Type', 'Total Students', 'Total School Budget',
       'Per Student Budget', 'Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', 'Overall Passing Rate',
       'Spending Bin', 'Spending Ranges (Per Student)', 'School Size'],
      dtype='object')

In [45]:
schools_by_type_df = school_summary_final.drop(school_summary_final.columns[[1, 2, 3, 9, 10, 11]], axis=1)
# schools_by_type_df = schools_by_bin_df.reset_index(drop=True)
schools_by_type_df.set_index('School Type', inplace=True)
schools_by_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
District,77.048432,81.033963,66.680064,81.933280,74.306672
Charter,83.061895,83.975780,94.133477,97.039828,95.586652
District,76.711767,81.158020,65.988471,80.739234,73.363852
District,77.102592,80.746258,68.309602,79.299014,73.804308
Charter,83.351499,83.816757,93.392371,97.138965,95.265668
District,77.289752,80.934412,66.752967,80.862999,73.807983
Charter,83.803279,83.814988,92.505855,96.252927,94.379391
District,76.629414,81.182722,65.683922,81.316421,73.500171
District,77.072464,80.966394,66.057551,81.222432,73.639992


In [46]:
schools_type_final = schools_by_type_df.groupby(schools_by_type_df.index)
schools_type_final.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
